In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
import torch

import sys
sys.path.append('../src/utils') 
from common import get_accuracy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SAVE_PATH = "/home/s/python_progs/DL_homeworks/deep-metric-ts-clustering/notebooks/model"

In [2]:
# Select diverse stocks from different sectors
tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'JPM', 'JNJ', 'XOM', 'NVDA', 'WMT']
ticker_to_class = {ticker: i for i, ticker in enumerate(tickers)}

data = yf.download(tickers, start="2020-01-01", end="2024-01-01")['Close']
data.dropna(inplace=True)
print(data)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed

Ticker            AAPL        AMZN       GOOGL         JNJ         JPM  \
Date                                                                     
2020-01-02   72.620834   94.900497   68.108376  126.055138  120.733566   
2020-01-03   71.914810   93.748497   67.752075  124.595726  119.140312   
2020-01-06   72.487854   95.143997   69.557945  124.440300  119.045601   
2020-01-07   72.146927   95.343002   69.423592  125.200226  117.021736   
2020-01-08   73.307503   94.598503   69.917725  125.182991  117.934631   
...                ...         ...         ...         ...         ...   
2023-12-22  192.192551  153.419998  140.816757  149.492691  161.660004   
2023-12-26  191.646561  153.410004  140.846634  150.146591  162.616074   
2023-12-27  191.745819  153.339996  139.702087  150.348541  163.591431   
2023-12-28  192.172699  153.380005  139.562759  150.569702  164.460587   
2023-12-29  191.130310  151.940002  139.025330  150.723557  164.267441   

Ticker            MSFT       NVDA    

In [3]:
def create_windows(series, window_size=30, step=1):
    windows = []
    for i in range(0, len(series) - window_size + 1, step):
        window = series[i:i + window_size].values
        windows.append(window)
    return windows


def create_dataset(windows):
    dataset = []
    for window in windows:

        dataset.append((window['window'], ticker_to_class[window['ticker']]))

    return dataset


In [4]:
window_size = 30
step = 5  # reduces overlap between samples
stock_windows = []  # Dictionary of ticker → list of windows

for ticker in tickers:
    for window in create_windows(data[ticker], window_size, step):
        stock_windows.append({
                'ticker': ticker,
                'window': window
            })
print(stock_windows[0])

{'ticker': 'AAPL', 'window': array([72.62083435, 71.91481018, 72.487854  , 72.14692688, 73.30750275,
       74.86463928, 75.03386688, 76.63692474, 75.6020813 , 75.27806091,
       76.22105408, 77.06490326, 76.5426178 , 76.81585693, 77.18579865,
       76.96334839, 74.70020294, 76.81342316, 78.42131042, 78.30766296,
       74.83559418, 74.63008881, 77.09391022, 77.72254181, 78.63166046,
       77.56287384, 77.93125153, 77.46105194, 79.30059052, 78.73588562])}


In [5]:
dataset = create_dataset(stock_windows)

In [6]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split

class StockDataset(Dataset):
    def __init__(self, stock_windows):
        self.stock_windows = stock_windows
        self.labels = [label for _, label in stock_windows]  # Extract and store labels

    def __len__(self):
        return len(self.stock_windows)

    def __getitem__(self, idx):
        x, label = self.stock_windows[idx]
        x = torch.tensor(x, dtype=torch.float32).unsqueeze(0)  # shape [1, 30]
        label = torch.tensor(label, dtype=torch.float32)
        return x, label

dataset = StockDataset(dataset)


train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size


train_dataset, val_dataset = random_split(dataset, [train_size, val_size])



In [7]:
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, input_dim, encoder_type, embedding_dim=128):
        super().__init__()

        self.encoder_type = encoder_type
        self.embedding_dim = embedding_dim


        if encoder_type=="CNN":
            self.conv_layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            )
            # Calculate final output size after conv and pooling layers based on input length
            self.fc = nn.Linear(32 * (input_dim // 4), embedding_dim)


        if encoder_type=="GRU":
            self.num_layers = 5
            self.hidden_size = 256
            self.gru = nn.GRU(1, self.hidden_size, self.num_layers, batch_first=True)
            self.fc = nn.Linear(self.hidden_size, self.embedding_dim)

        if encoder_type=="LSTM":    
            self.num_layers = 2
            self.hidden_size = 256
            self.lstm = nn.LSTM(1, self.hidden_size, self.num_layers, batch_first=True)
            self.fc = nn.Linear(self.hidden_size, self.embedding_dim)

    def forward(self, x):
        if self.encoder_type=="CNN":
            # x = x.unsqueeze(1)  # add channel dim: (batch_size, 1, seq_len)
            x = self.conv_layers(x)
            x = x.view(x.size(0), -1)  # flatten
            x = self.fc(x)
        
        if self.encoder_type=="GRU":
            x = x.reshape(-1, window_size, 1)

            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

            x, _ = self.gru(x, h0)
            x = x[:, -1, :]
            x = self.fc(x)

        if self.encoder_type=="LSTM":
            x = x.reshape(-1, window_size, 1)

            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)


            x, _ = self.lstm(x, (h0,c0))
            x = x[:, -1, :]
            x = self.fc(x)


        return x




In [8]:

from torch.utils.data import DataLoader
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
mining_func = miners.TripletMarginMiner(margin=0.2, distance=distance, type_of_triplets="semihard")
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)

# model = Encoder(window_size, 'CNN')
# model = Encoder(window_size, 'LSTM')
model = Encoder(window_size, 'GRU')
# model = Encoder(window_size, 'TRANSFORMER')

model.to(device)

Encoder(
  (gru): GRU(1, 256, num_layers=5, batch_first=True)
  (fc): Linear(in_features=256, out_features=128, bias=True)
)

In [10]:
import torch
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
import torch.optim as optim
from tqdm import tqdm
import json

batch_size = 512

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
history = {"train": [], "val": [], "best_accuracy": 0.0}

scaler = GradScaler()  # Helps manage fp16 gradients

optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 1000

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for x, label in train_loader:
        x, label = x.to(device), label.to(device)

        optimizer.zero_grad()

        with autocast(dtype=torch.float16, device_type=device.type): 
            embeddings = model(x)
            indices_tuple = mining_func(embeddings, label)
            loss = loss_func(embeddings, label, indices_tuple)

        # Backpropagation with scaled gradients
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()




    if epoch %10 == 0:
        # model validation
        model.eval()
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
        
        with torch.no_grad():
            # as all embeddings need to be stored in memory
            # you can set DEVICE = torch.device('cpu') in case gpu memory overflow occurs
            accuracy = get_accuracy(val_dataset, train_dataset, model, device.type)
            
            history["val"].append({"epoch": epoch, "accuracy": accuracy})
            print(f"Val accuracy: {accuracy}")
            
            # save model
            torch.save(model.state_dict(), f"{SAVE_PATH}/model_latest.pth")
            
            if accuracy >= history["best_accuracy"]:
                history["best_accuracy"] = accuracy
                torch.save(model.state_dict(), f"{SAVE_PATH}/model_best.pth")

            with open("history.json", "w") as f:
                f.write(json.dumps(history))

Epoch 1/1000, Loss: 0.0956


100%|██████████| 49/49 [00:00<00:00, 290.14it/s]


Val accuracy: 0.6632653061224489
Epoch 11/1000, Loss: 0.0844


100%|██████████| 49/49 [00:00<00:00, 235.16it/s]


Val accuracy: 0.8341836734693877
Epoch 21/1000, Loss: 0.0945


...

Val accuracy: 0.8979591836734694
Epoch 981/1000, Loss: 0.0353


100%|██████████| 49/49 [00:00<00:00, 385.67it/s]


Val accuracy: 0.8698979591836735
Epoch 991/1000, Loss: 0.0488


100%|██████████| 49/49 [00:00<00:00, 311.30it/s]


Val accuracy: 0.875


In [9]:

model.load_state_dict(torch.load("/home/s/python_progs/DL_homeworks/deep-metric-ts-clustering/notebooks/model/model_GRU_absolute_best.pth", map_location=torch.device("cuda")))
model.to("cuda")

model.eval()

with torch.no_grad():
    # as all embeddings need to be stored in memory
    # you can set DEVICE = torch.device('cpu') in case gpu memory overflow occurs
    accuracy = get_accuracy(val_dataset, train_dataset, model, device.type)
    
    print(f"Val accuracy: {accuracy}")

100%|██████████| 49/49 [00:00<00:00, 380.05it/s]

Val accuracy: 0.9821428571428571
